In [ ]:
from src.classify3D import generate_data, split_data

In [ ]:
from importlib import reload
reload(split_data)

In [ ]:
gen1 = generate_data.GenerateData(radial_scale=1)

In [ ]:
gen1('notebook')

In [ ]:
gen1.X.shape

In [ ]:
gen1.y.shape

In [ ]:
data = split_data.SplitData(gen=gen1)

In [ ]:
data.X

In [ ]:
data.y

In [ ]:
gen2 = generate_data.GenerateData(radial_scale=2)

In [ ]:
gen2('notebook')

In [ ]:
gen2.X.shape

In [ ]:
gen2.y.shape

In [ ]:
split2 = split_data.SplitData(gen=gen2)

In [ ]:
split2.X

In [ ]:
split2.y

Build a class which transforms an X by various scales. Takes in a
split X, that is, X where X has X.train, X.test, X.val components. If X
is just an array it treats it like that, but if it has components it looks for
the existence of train test and val. Returns an object that is
X.transformation if it took in just an array, or X.transformation.train, test,
and val if those existed. Also returns the scaler objects used as
scaler.transformation for future transformations as necessary. X.train and such
are also returns in addition to the transformations.

Build a class which performs PCA on an X, taking in a parameter to tell how
many features to create. By default 3 for chartability. If X is just an array,
it performs PCA. Otherwise it looks for train test val and transformations to
perform PCA at each step of the way. Returns X.pca.transformation.train, etc.

Builds a class which creates a variety of graphics about the train set. Basic
statistics, class by class break downs of basic statistics, correlation between
features, correlation between features class by class, heat map, heat map class
by class, and class distributions. Returns a display object eda.train, which
has various graphs attached to it.

Builds a class which creates a variety of graphics about the test and
validation set. Overall basic statistics, correlation between features, heat
map, novelty detection counts by method, novelty detection by method. Returns
a display object eda.test and eda.val, which has graphs attached. Can accept an
eda object to modify, or creates one new.